# PyTorchの勉強用 
- PyTorch公式チュートリアル(https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)を和訳

## 目的（なぜやるのか）
- PyTorchの基礎的な概念を学習する

## パッケージのimportとPyTorchのバージョンを確認

In [1]:
# パッケージのimport
import numpy as np
import json
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
from torchvision import models, transforms


In [2]:
# PyTorchのバージョン確認
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)


PyTorch Version:  1.5.0
Torchvision Version:  0.6.0


# PyTorch公式チュートリアル
## PyTorchについて

pytorchのコアとなる機能は、主に2つある
- n次元テンソル(Tensor)：numpyに似ているが、GPU上でも動作する
- 自動微分(Automatic differentiation)：ニューラルネットワークの構築・訓練に用いることができる



## テンソル(Tensor)

### numpyのウォームアップ

PyTorchを用いる前に、まずはnumpyを用いてネットワークを実装する

numpyはn次元の配列オブジェクトと、その配列を操作するための多くの関数を提供している。numpyは計算科学のための汎用的なフレームワークであり、計算グラフ(computation graphs)、ディープラーニング、勾配の概念については特に考えられていない。しかし、2層のニューラルネットワークをランダムなデータに適合させる上で、numpyの演算を使ってネットワークの順伝播・逆伝播のパスを手動で実装することで、numpyでもこれらの概念を実装できる

In [3]:
# N：バッチ数 
# D_in：入力層の次元数
# H：隠れ層の次元数
# D_out：出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# ランダムな入力・出力データの作成
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 重みをランダムな値で初期化
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

# 学習率の設定
learning_rate = 1e-6

for t in range(500):
    # 順伝播：yの予測値を計算
    h = x.dot(w1) # 入力に重みw1をかけて隠れ層に渡す
    h_relu = np.maximum(h, 0)# reLU関数で活性化
    y_pred = h_relu.dot(w2) # 重みw2をかけてyの予測値を算出

    # 損失の計算
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # 逆伝播：損失を元に重みw1, w2の勾配を計算
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # 重みの更新
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 54384746.05910431
1 61313253.855912626
2 59508280.3641528
3 38322748.59168631
4 16196244.449533762
5 5841402.904879023
6 2850699.193637717
7 1925162.8143797358
8 1501905.6937964123
9 1228933.6185847118
10 1024573.13631874
11 863622.6656617752
12 734249.5826519736
13 628541.5677758548
14 541238.7188067418
15 468533.11262210057
16 407557.61121277383
17 356165.37801844906
18 312495.85045912926
19 275213.39513825555
20 243180.57273486073
21 215545.91751103048
22 191598.7752599609
23 170785.4188023764
24 152631.22350790192
25 136727.61170439963
26 122761.4496576972
27 110478.92072843657
28 99637.28630715127
29 90037.6135748188
30 81518.38416239197
31 73945.13313887116
32 67190.0102672246
33 61151.695713335794
34 55746.14070072501
35 50896.79661547007
36 46542.027985831475
37 42623.371920488775
38 39090.244845010006
39 35899.47144501709
40 33013.59836808576
41 30398.86510324382
42 28025.306404904666
43 25867.963559783977
44 23904.544956979516
45 22116.977833585916
46 20486.04205059912
47 1

367 0.11611362090064145
368 0.11261335387359989
369 0.10921944046574383
370 0.1059282741697617
371 0.10273726164627295
372 0.09964278877129149
373 0.09664202201881575
374 0.09373255046551235
375 0.0909110076641462
376 0.08817489634581135
377 0.08552192121469035
378 0.08294891718979958
379 0.0804539840940541
380 0.07803469259521438
381 0.0756883001162931
382 0.07341309535645071
383 0.0712066213433411
384 0.06906675631846951
385 0.0669918407618617
386 0.06497936617572554
387 0.06302777848128144
388 0.061135311531690245
389 0.05929979623173878
390 0.057519870715624546
391 0.0557935722597942
392 0.054119274710928415
393 0.05249570237043546
394 0.0509209323117797
395 0.04939368438771351
396 0.047912619958565154
397 0.046476076826945295
398 0.04508289502388513
399 0.04373167816329767
400 0.04242109865757906
401 0.041150162463895655
402 0.03991735105393476
403 0.03872170796232814
404 0.03756215233651696
405 0.03643779992454874
406 0.03534852877831607
407 0.03429038057138928
408 0.033264036542

### PyTorch：Tensor

numpyは素晴らしいフレームワークであるが、GPUを用いて数値計算を高速化することはできない。最近のディープニューラルネットワークでは、GPUで50倍以上の高速化が可能な場合が多いため、残念ながらnumpyだけでは不十分である

ここで、PyTorchの最も基本的な概念であるテンソル(Tensor)を紹介する。PyTorchのテンソルは、概念的にはnumpy配列と同じである。しかし、numpy配列に加えて計算グラフや勾配の追跡も可能である他、GPUを利用して数値計算を高速化することが可能である。GPU上でPyTorch Tensorを実行するには、新しいデータ型にキャストする必要がある

以下では、上記のnumpyの例と同じようにランダムなデータに2層のネットワークをフィットさせる際、PyTorch Tensorを用いている。ここで、ネットワークの順伝播・逆伝播のパスも同様に手動で実装する必要がある

In [22]:
dtype = torch.float
# device = torch.device("cpu")
device = torch.device("cuda:0") # GPUを用いる場合はこの文をアンコメント

# N：バッチ数 
# D_in：入力層の次元数
# H：隠れ層の次元数
# D_out：出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# ランダムな入力・出力データの作成
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 重みをランダムな値で初期化
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

# 学習率の設定
learning_rate = 1e-6
for t in range(500):
    # 順伝播：yの予測値を計算
    h = x.mm(w1) # mm：2次元の行列同士の積を計算する関数
    h_relu = h.clamp(min=0) # clamp(input, min, max)：inputのすべての要素について、[min,max]の範囲外の値をそれぞれmin,maxに変換する関数
    y_pred = h_relu.mm(w2)

    # 損失の計算
    loss = (y_pred - y).pow(2).sum().item() # pow(exponent):exponentの値で累乗する関数
    if t % 100 == 99: # 100ステップごとに損失をプリント
        print(t, loss)

    # 逆伝播：損失を元に重みw1, w2の勾配を計算
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # 勾配降下を用いて重みの更新
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 308.6850891113281
199 0.7617379426956177
299 0.002684763167053461
399 8.705579239176586e-05
499 2.1205840312177315e-05


## autograd

### PyTorch：Tensorとautograd
上記の例ではニューラルネットワークの順伝播・逆伝播のパスを手動で実装していた。しかし、大規模で複雑なネットワークでは同じことをしようとすると非常に手間がかかってしまう。

ここで、PyTorchでは自動微分(automatic differentiation)を用いてニューラルネットワークの逆伝播の計算を自動化することができる。PyTorchのautogradパッケージがこの機能を持つ。autogradを用いると、ネットワークの順伝播パスにて計算グラフが定義される。グラフのノードはテンソルで、エッジは入力テンソルから出力テンソルを生成する関数である。このグラフに対して誤差逆伝播を行うことで、勾配を簡単に計算することができる

これは一見複雑に聞こえるが、実際には非常に簡単に用いることができる。各テンソルは計算グラフのノードを表し、あるテンソルxがx.require_grad=Trueのテンソルであれば、x.gradはスカラー値に対するxの勾配を保持する別のテンソルである

以下では、PyTorch Tensorとautogradを用いて2層ネットワークを実装している

In [5]:
dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # GPUを用いる場合はこの文をアンコメント

# N：バッチ数 
# D_in：入力層の次元数
# H：隠れ層の次元数
# D_out：出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# ランダムな入力・出力データの作成
# requires_grad=Falseに設定することで、逆伝播の際そのテンソルに関する勾配は計算しなくなる
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 重みをランダムな値で初期化
# requires_grad=Trueに設定することで、逆伝播の際にそのテンソルに関する勾配を計算するようになる
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 順伝播：これは先程のTensorsを用いた順伝播パスの計算に用いたものと全く同じ
    # しかし、逆伝播を手作業で実装していないので、中間値を保持する必要がない
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # テンソルの演算を用いて損失を計算
    # この時、損失はテンソルのshape (1,)に保持されている
    # loss.item()はlossに保持しているスカラー値を取得する関数
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # autograd を使用して逆伝播を計算する。この呼び出しでは、requires_grad=True のすべてのテンソルについて損失の勾配を計算している
    #この呼び出しの後、w1.gradとw2.gradはそれぞれw1とw2に対する損失の勾配を保持するテンソルとなる
    loss.backward()

    # 勾配降下を用いて手動で重みの更新を行い torch.no_grad()でラップする
    # 重みのテンソルはrequires_grad=Trueであるため、手動で追跡する必要はない
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 重みを更新した後、手動で勾配をゼロにする
        w1.grad.zero_()
        w2.grad.zero_()

99 621.9561157226562
199 5.948866367340088
299 0.08107059448957443
399 0.0014935587532818317
499 0.00011179103603353724


### PyTorch：新しいautograd関数の定義
それぞれの原始的なautograd演算子は、実際にはテンソルで動作する2つの関数である。forward関数は入力テンソルから出力テンソルを計算する。backward関数はスカラー値に対する出力テンソルの勾配を受け取り、同じスカラー値に対応する入力テンソルの勾配を計算する

PyTorchでは、torch.autograd.Functionのサブクラスを定義し、forward関数とbackward関数を実装することで、簡単に独自のautograd演算子を定義することができる。インスタンスを作成して関数のように呼び出し、入力データを含むテンソルを渡すことで、autograd演算子を使用することができる

以下の例では、ReLU非線形性を実行するための独自のautograd関数を定義し、それを用いて2層のネットワークを実装している

In [6]:
class MyReLU(torch.autograd.Function):
    """
    torch.autograd.Functionをサブクラス化し、
    テンソル上で動作する順伝播パスと逆伝播パスを実装することで、
    独自のautograd Functionsを実装できる
    """

    @staticmethod
    def forward(ctx, input):
        """
        順伝播パスでは、入力を含むテンソルを受け取り、出力を含むテンソルを返す
        ctxは、逆伝播の計算のための情報を格納するために用いることのできるコンテキストオブジェクトである
        ctx.save_for_backwardメソッドを使用することで、任意のオブジェクトをキャッシュして逆伝播パスにて用いることができる
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        逆伝播パスでは、出力に対する損失の勾配を受け取り、
        そこから入力に対する損失の勾配を計算する必要がある
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # GPUを用いる場合はこの文をアンコメント

# N：バッチ数 
# D_in：入力層の次元数
# H：隠れ層の次元数
# D_out：出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# ランダムな入力・出力データの作成
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 重みをランダムな値で初期化
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 関数を適用するには、Function.applyメソッドを用いる。今回はこれを'relu'とエイリアスする
    relu = MyReLU.apply

    # 順伝播：カスタムしたautograd演算を用いてReLUを計算し、それを用いて予測されたyを計算する
    y_pred = relu(x.mm(w1)).mm(w2)

    # 損失を計算し、print
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # autograd を使用して逆伝播を計算
    loss.backward()

    # 勾配降下を用いて重みの更新
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 重みを更新した後、手動で勾配をゼロにする
        w1.grad.zero_()
        w2.grad.zero_()

99 259.4796447753906
199 0.5658110976219177
299 0.0022444366477429867
399 9.172491263598204e-05
499 2.313937511644326e-05


## nnモジュール

### PyTorch：nn
計算グラフとautogradは複雑な演算子を定義し、自動的に導関数を得るうえで非常に強力なパラダイムである。しかしながら、大規模なニューラルネットワークの場合、生のautogradは少し低レベルすぎるかもしれない

ニューラルネットワークを構築する際には、計算を層に配置することをよく考えるが、その中には学習可能なパラメータがあり、学習中に最適化される

TensorFlowでは、Keras, TensorFlow-Slim, TFLearnなどのパッケージが、ニューラルネットワークを構築するうえで便利な、生の計算グラフ上の高レベルの抽象化を提供している

PyTorchでは、nnパッケージがこれと同じ目的を果たしている。nnパッケージは、ニューラルネットワーク層とほぼ同等のモジュールを定義している。モジュール入力テンソルを受け取り、出力テンソルを計算するが、学習可能パラメータを含むテンソルのような内部状態を保持することもできる。また、nnパッケージは、ニューラルネットワークを学習する際に一般的に用いられる有用な損失関数のセットも定義している

以下の例では、nnパッケージを用いて2層ネットワークを実装している

In [7]:
# N：バッチ数 
# D_in：入力層の次元数
# H：隠れ層の次元数
# D_out：出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# ランダムな入力・出力データの作成
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nn パッケージを使用して、モデルを層のシーケンスとして定義する
# nn.Sequentialは、他のモジュールを含み、それらを順番に適用して出力を生成するモジュールである
# 各Linearモジュールは、線形関数を使って入力から出力を計算し、重みとバイアスのための内部テンソルを保持する
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# nnパッケージには、一般的な損失関数の定義も含まれている。今回は損失関数として平均二乗誤差(MSE)を使用
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # 順伝播：xをモデルに渡すことでyの予測値を計算する
    # モジュールオブジェクトは__call__演算子をオーバーライドするので、関数のように呼び出すことができる
    # 入力データのテンソルをモジュールに渡すと出力データのテンソルが生成される
    y_pred = model(x)

    # 損失を計算してprintする
    # loss_fnにyの予測値のテンソルと真値のテンソルを渡すことで、損失を含むテンソルが出力される
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 逆伝播を実行する前に勾配をゼロにする
    model.zero_grad()

    # 逆伝播：モデルのすべての学習可能パラメータに対する損失の勾配を計算する
    # 内部では、各モジュールのパラメータはrequires_grad=Trueでテンソルに格納されているので、
    # この呼び出しではモデル内のすべての学習可能なパラメータの勾配を計算する
    loss.backward()

    # 勾配降下を用いて重みを更新する。各パラメータはテンソルであるため、先程と同じくその勾配にアクセスできる
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 2.0699803829193115
199 0.04423416778445244
299 0.0021148109808564186
399 0.0001349154335912317
499 9.769574717211071e-06


### PyTorch: optim
ここまでは、学習可能なパラメータを保持しているテンソルを手動で変異させることでモデルの重みを更新していた（autogradによる過去の値の追跡を避けるため、torch.no_grad()や.dataを使用していた）。これは確率的勾配降下のような単純な最適化アルゴリズムでは大きな負担とはならないが、実際にはAdaGrad, RMSProp, Adamなどのより高度な最適化アルゴリズムを用いてニューラルネットワークを訓練することがよくある

PyTorchのoptimパッケージでは、最適化アルゴリズムの概念を抽象化し、一般的に用いられている最適化アルゴリズムの実装を提供している

以下の例では、一つ上と同様にnnパッケージを用いてモデルを定義しているが、optimパッケージが提供するAdamアルゴリズムを用いてモデルを最適化している

In [8]:
# N：バッチ数 
# D_in：入力層の次元数
# H：隠れ層の次元数
# D_out：出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# ランダムな入力・出力データの作成
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nnパッケージを用いてモデルと損失関数を定義
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# optimパッケージを使用して、モデルの重みを更新する最適化アルゴリズムを定義する
# ここではAdamを使用する。optimパッケージには他にも多くの最適化アルゴリズムが含まれている
# Adam コンストラクタの最初の引数は、最適化アルゴリズムがどのテンソルを更新するかを指定する
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # 順伝播：xをモデルに渡すことでyの予測値を計算する
    y_pred = model(x)

    # 損失を計算してprint
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 逆伝播を行う前に、optimizerオブジェクトを用いて更新する変数（モデルの学習可能な重み）の勾配をすべてゼロにする
    # これは、デフォルトでは.backward()が呼び出される度に勾配がバッファに蓄積される（つまり上書きされない）ためである
    # 詳細はtorch.autograd.backwardの解説を参照
    optimizer.zero_grad()

    # 逆伝播：モデルパラメータに対する損失の勾配を計算する
    loss.backward()

    # optimizerのステップ関数を呼び出すと、そのパラメータが更新される
    optimizer.step()

99 32.187049865722656
199 0.47134101390838623
299 0.004644058644771576
399 1.6544943719054572e-05
499 2.2352482531573514e-08


### PyTorch：nnモデルのカスタム

時には、既存のモジュールのシーケンスよりも複雑なモデルを指定したいこともあるだろう。このような場合には、nn.Moduleをサブクラス化し、入力テンソルを受け取り、テンソル演算のための他のモジュールや他のautogradを生成する順伝播を定義することで、独自のモジュールを定義することができる

以下の例では、2層のネットワークをカスタムモジュールのサブクラスとして実装している

In [9]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        このコンストラクタでは、2つのnn.Linearモジュールをインスタンス化し、メンバ変数として割り当てる
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        順伝播関数では、入力データのテンソルから出力データのテンソルを返さなければならない
        ここで、コンストラクタで定義されたモジュールや、テンソルに対する任意の演算子を用いることができる
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred
    
# N：バッチ数 
# D_in：入力層の次元数
# H：隠れ層の次元数
# D_out：出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# ランダムな入力・出力データの作成
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 上記で定義したクラスをインスタンス化してモデルを構築する
model = TwoLayerNet(D_in, H, D_out)

# 損失関数と最適化アルゴリズムを定義する
# SGDコンストラクタのmodel.parameters() の呼び出しには、モデルのメンバ変数である
# 2つのnn.Linearモジュールの学習可能なパラメータが含まれる
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # 順伝播：xをモデルに渡すことでyの予測値を計算する
    y_pred = model(x)

    # 損失を計算してprint
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 勾配をゼロにし、逆伝播を実行し、重みを更新する
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 2.3383655548095703
199 0.04339797794818878
299 0.0017115985974669456
399 9.253511962015182e-05
499 6.024424692441244e-06


### PyTorch：フロー制御＋重み共有

動的モデルと重み共有の例として、非常に奇妙なモデルを実装する
完全に接続されたReLUネットワークは、各順伝播パスにて1から4の間の乱数を選択し、その数ぶんの隠れ層を用い、最内層の隠れ層を計算するために同じ重みを複数回再利用する

このモデルでは、ループを実装する上で通常のPythonのフロー制御を用いることができ、順伝播パスを定義する際に同じモジュールを複数回再利用することで、最内層での重み共有を実装することができる

このモデルは以下のようにモジュールのサブクラスとして簡単に実装ができる

In [10]:
import random


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        コンストラクタでは、順伝播パスで使用する 3 つの nn.Linearインスタンスを構築する
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        モデルの順伝播パスでは、0,1,2,3のいずれかをランダムに選択し、
        middle_linearモジュールを何度も再利用して、隠れ層の表現を計算する
        
        各順伝播パスは動的な計算グラフを構築するため、モデルの順伝播パスを定義する際に
        ループや条件文のような通常のPythonのフロー制御演算子を使用することができる
        
        ここでは、計算グラフを定義する際に同じモジュールを何度も再利用しても全く問題ないことがわかる
        これは各モジュールを一度しか使用できなかったLua Torchからの大きな改善である
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred
    
# N：バッチ数 
# D_in：入力層の次元数
# H：隠れ層の次元数
# D_out：出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# ランダムな入力・出力データの作成
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 上記で定義したクラスをインスタンス化してモデルを構築する
model = DynamicNet(D_in, H, D_out)

# 損失関数と最適化アルゴリズムを構築する
# この奇妙なモデルをバニラの確率的勾配降下法で学習するのは難しいため、今回はmomentumを用いる
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # 順伝播：xをモデルに渡すことでyの予測値を計算する
    y_pred = model(x)

    # 損失を計算してprint
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 勾配をゼロにし、逆伝播を実行し、重みを更新する
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 47.88127136230469
199 2.7229580879211426
299 2.4152848720550537
399 0.48642584681510925
499 0.11632964015007019


In [20]:
print(torch.cuda.is_available())

True


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
x = torch.rand(5,3, requires_grad=True)
x

tensor([[0.8699, 0.3236, 0.9675],
        [0.2016, 0.1074, 0.6263],
        [0.4429, 0.7860, 0.8440],
        [0.0281, 0.0070, 0.1349],
        [0.9484, 0.4282, 0.2927]], requires_grad=True)

In [14]:
y = x * x

In [15]:
y

tensor([[7.5665e-01, 1.0469e-01, 9.3603e-01],
        [4.0656e-02, 1.1537e-02, 3.9228e-01],
        [1.9616e-01, 6.1779e-01, 7.1225e-01],
        [7.9006e-04, 4.8473e-05, 1.8189e-02],
        [8.9940e-01, 1.8334e-01, 8.5687e-02]], grad_fn=<MulBackward0>)

In [16]:
out = x.mean()

In [17]:
out

tensor(0.4672, grad_fn=<MeanBackward0>)

In [18]:
out.backward()

In [19]:
x.grad

tensor([[0.0667, 0.0667, 0.0667],
        [0.0667, 0.0667, 0.0667],
        [0.0667, 0.0667, 0.0667],
        [0.0667, 0.0667, 0.0667],
        [0.0667, 0.0667, 0.0667]])

In [18]:
x = torch.randn(3, requires_grad=True)
x

tensor([-0.0208, -0.3717,  0.0967], requires_grad=True)

In [19]:
y = x * 2
while y.data.norm() < 1000:
    y = y * 2 
    
print(y)

tensor([  -85.2232, -1522.4877,   396.1404], grad_fn=<MulBackward0>)


In [20]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([4.0960e+02, 4.0960e+03, 4.0960e-01])
